# Coding Challenge
## Image Median Filter Parallelization

### Description of the Challenge
Local median filter is used for smoothing images and noise reduction. The degree of smoothing is determined by the kernel size which can be 2D or 3D. Applying 3D median filter to large 3D images (eg. 2000x2000x2000 voxels) is challenging because the images often do not fit in the memory and single-threaded execution of the filter is inefficient and too slow.


### Task 1
Create a 3D synthetic image and split it into subvolumes and apply median filter (like skimage.filters.median) to each subvolume in parallel. The output needs to be a 3D array with the same size as the original 3D image.

In [ ]:
from skimage.io import imread
from skimage.filters import median

### Dask-based solution

In [25]:
from dask.distributed import Client
from dask.array.image import imread as imread_dask
from dask_image.ndfilters import median_filter
import dask
import numpy as np
import napari



In [22]:
imread("cells3d.tif").shape

(60, 2, 256, 256)

In [21]:
img = imread_dask("cells3d.tif")
print(img.shape)
img

(1, 60, 2, 256, 256)


dask.array<imread, shape=(1, 60, 2, 256, 256), dtype=uint16, chunksize=(1, 60, 2, 256, 256), chunktype=numpy.ndarray>

This image has unusual dimensions, but we can look at it with napari using the following code:
```python
v = napari.Viewer()
v.add_image(img)
```
We extract the second spectral channel like this

In [23]:
stack = img[0, :, 1, :, :]

In [24]:
stack

dask.array<getitem, shape=(60, 256, 256), dtype=uint16, chunksize=(60, 256, 256), chunktype=numpy.ndarray>

In [19]:
v = napari.Viewer()
v.add_image(img)

C:\Users\rkiselev\Anaconda3\envs\fret\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'img' at 0x144362b3ac0>

In [29]:
filter_size = (3, 1, 1)
filtered = median_filter(stack, size=filter_size)
v = napari.Viewer()
v.add_image(filtered)
v.add_image(stack)

C:\Users\rkiselev\Anaconda3\envs\fret\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'stack' at 0x14436215a00>